# Title : Analyzing Weather Data from OpenWeatherMap API

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.plugins import HeatMap

In [ ]:
api_key = '373256372741af36710261b69fb037e4' 

In [ ]:
lat = 35.4639
lon = 77.0439

In [ ]:
api_url = f"http://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={api_key}"

In [ ]:
response = requests.get(api_url)
weather_data = response.json()
weather_data.keys()
len(weather_data['list'])
weather_data['list'][0]

In [ ]:
timestamps = [pd.to_datetime(item['dt'], unit='s') for item in weather_data['list']]
temperatures = [item['main']['temp'] for item in weather_data['list']]
humidity = [item['main']['humidity'] for item in weather_data['list']]
wind_speed = [item['wind']['speed'] for item in weather_data['list']]
weather_description = [item['weather'][0]['description'] for item in weather_data['list']]

In [ ]:
weather_df = pd.DataFrame({'Timestamp': timestamps, 
                           'Temperature': temperatures, 
                           'humidity': humidity, 
                           'wind_speed':wind_speed,
                           'weather_description': weather_description})

In [ ]:
weather_df.set_index('Timestamp', inplace=True)

weather_df['Temperature'] = weather_df['Temperature'].apply(lambda x: x - 273.15 if isinstance(x, float)else x)

weather_df.fillna(method='ffill', inplace=True) 
 
display(weather_df.head())
print(weather_df.dtypes)
display(weather_df.describe(exclude=['datetime','object']))

In [ ]:
daily_mean_temp = weather_df['Temperature'].resample('D').mean()
daily_mean_humidity = weather_df['humidity'].resample('D').mean()
daily_mean_wind_speed = weather_df['wind_speed'].resample('D').mean()

In [ ]:
daily_mean_temp.plot(kind='line')
plt.title('Mean Daily Temperature')
plt.xlabel('Date')
plt.ylabel('Temperature')
plt.grid(True)
plt.show()

daily_mean_humidity.plot(kind='bar')
plt.title('Mean Daily Humidity')
plt.xlabel('Date')
plt.ylabel('Humidity')
plt.grid(True)
plt.show()

plt.scatter(weather_df['Temperature'], weather_df['wind_speed'])
plt.title('Temperature vs. Wind Speed')
plt.xlabel('Temperature')
plt.ylabel('Wind Speed')
plt.grid(True)
plt.show()

sns.heatmap(weather_df[['Temperature', 'humidity']],annot=True)
plt.title('Temperature vs Humidity')
plt.show()

plt.scatter(weather_df['Temperature'], weather_df['humidity'])
plt.xlabel('Temperature')
plt.ylabel('Humidity')
plt.title('Temperature vs Humidity')
plt.show()

In [ ]:
map = folium.Map(location=[lat, lon], zoom_start=12)

heatmap_data = [[lat, lon, weather_df['Temperature'].iloc[0]]]  

folium.Marker(
    location=[lat, lon],
    popup=f"Temperature: {weather_df['Temperature'].iloc[0]:.2f}°C<br>Humidity: {weather_df['humidity'].iloc[0]}%<br>{weather_description[0]}",
    icon=folium.Icon(color='blue')
).add_to(map)

HeatMap(heatmap_data, min_opacity=0.2, radius=15, blur=20, max_zoom=18).add_to(m)

map.save("weather_map.html")

map